# Instructions

Install Environment
  conda env create -f brainPlot2.yml

  If anything fails and you need to run it again to update it, use the following command
    conda env update -n brainPlot2 -f nora.yml
  If gnu readlines fails, you can just remove the whole line from the yml file and run the update command
  If an existing installation of vtk is already installed by your system, you can 
    1) remove it from you system and run the update command
    2) remove it from the yml file (if the version is 8.2.0) and run the update command

Set up file structure
  [Download lh.vtk into folder]
  [Download rh.vtk into folder]
  mkdir coords
  mv catFR1_HPC_ca1_sub_channel_t-scores-min-25_trials.npz coords

Build the diagram
  conda activate brainPlot2
  python brainMakerRipples.py

# Includes

In [6]:
import warnings
import pandas as pd

def ignore_warnings(func):
    def wrapper(*args, **kwargs):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            return func(*args, **kwargs)
    return wrapper

def show_all_pandas_columns(func):
    def wrapper(*args, **kwargs):
        old_max = pd.options.display.max_columns
        pd.options.display.max_columns = None
        ret = func(*args, **kwargs)
        pd.options.display.max_columns = old_max
        return ret
    return wrapper

In [13]:
# 2021-01-14 John Sakon - How to load pairs using readers and get the regions for each pair from our updated pipeline
# 2021-11-03 James Bruska - Updated to import copy (which is needed)

from copy import copy
from typing import Optional
from collections import namedtuple
from dataclasses import dataclass
from cmlreaders import CMLReader

@dataclass
class CMLSubjectArgs:
    subject: str
    localization: int = 0
    montage: int = 0
    session: int = 1
    experiment: str = ""
    def __iter__(self):
        return iter((self.subject, self.localization, self.montage, self.session, self.experiment))


def Loc2PairsTranslation(pairs,localizations):
    # localizations is all the possible contacts and bipolar pairs locations
    # pairs is the actual bipolar pairs recorded (plugged in to a certain montage of the localization)
    # this finds the indices that translate the localization pairs to the pairs/tal_struct

    loc_pairs = localizations.type.pairs
    loc_pairs = np.array(loc_pairs.index)
    split_pairs = [pair.upper().split('-') for pair in pairs.label] # pairs.json is usually upper anyway but things like "micro" are not
    pairs_to_loc_idxs = []
    for loc_pair in loc_pairs:
        loc_pair = [loc.upper() for loc in loc_pair] # pairs.json is always capitalized so capitalize location.pairs to match (e.g. Li was changed to an LI)
        loc_pair = list(loc_pair)
        idx = (np.where([loc_pair==split_pair for split_pair in split_pairs])[0])
        if len(idx) == 0:
            loc_pair.reverse() # check for the reverse since sometimes the electrodes are listed the other way
            idx = (np.where([loc_pair==split_pair for split_pair in split_pairs])[0])
            if len(idx) == 0:
                idx = ' '
        pairs_to_loc_idxs.extend(idx)

    return pairs_to_loc_idxs # these numbers you see are the index in PAIRS frame that the localization.pairs region will get put


def get_elec_regions(localizations,pairs): 
    # 2020-08-13 new version after consulting with Paul 
    # suggested order to use regions is: stein->das->MTL->wb->mni
    
    # 2020-08-26 previous version input tal_struct (pairs.json as a recArray). Now input pairs.json and localizations.json like this:
    # pairs = reader.load('pairs')
    # localizations = reader.load('localization')
    
    regs = []    
    atlas_type = []
    pair_number = []
    has_stein_das = 0
    
    # if localization.json exists get the names from each atlas
    if len(localizations) > 1: 
        # pairs that were recorded and possible pairs from the localization are typically not the same.
        # so need to translate the localization region names to the pairs...which I think is easiest to just do here

        # get an index for every pair in pairs
        loc_translation = Loc2PairsTranslation(pairs,localizations)
        loc_dk_names = ['' for _ in range(len(pairs))]
        loc_MTL_names = copy(loc_dk_names) 
        loc_wb_names = copy(loc_dk_names)
        for i,loc in enumerate(loc_translation):
            if loc != ' ': # set it to this when there was no localization.pairs
                if 'atlases.mtl' in localizations: # a few (like 5) of the localization.pairs don't have the MTL atlas
                    loc_MTL_names[loc] = localizations['atlases.mtl']['pairs'][i] # MTL field from pairs in localization.json
                    has_MTL = 1
                else:
                    has_MTL = 0 # so can skip in below
                loc_dk_names[loc] = localizations['atlases.dk']['pairs'][i]
                loc_wb_names[loc] = localizations['atlases.whole_brain']['pairs'][i]   
    for pair_ct in range(len(pairs)):
        try:
            pair_number.append(pair_ct) # just to keep track of what pair this was in subject
            pair_atlases = pairs.iloc[pair_ct] #tal_struct[pair_ct].atlases
            if 'stein.region' in pair_atlases: # if 'stein' in pair_atlases.dtype.names:
                test_region = str(pair_atlases['stein.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region != 'nan'):
                    regs.append(test_region.lower())
#             if 'stein' in pair_atlases.dtype.names:  ### OLD WAY FROM TAL_STRUCT...leaving as example
#                 if (pair_atlases['stein']['region'] is not None) and (len(pair_atlases['stein']['region'])>1) and \
#                    (pair_atlases['stein']['region'] not in 'None') and (pair_atlases['stein']['region'] != 'nan'):
#                     regs.append(pair_atlases['stein']['region'].lower())
                    atlas_type.append('stein')
                    has_stein_das = 1 # temporary thing just to see where stein/das stopped annotating
                    continue # back to top of for loop
                else:
                    pass # keep going in loop
            if 'das.region' in pair_atlases:
                test_region = str(pair_atlases['das.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region != 'nan'):
                    regs.append(test_region.lower())
                    atlas_type.append('das')
                    has_stein_das = 1
                    continue
                else:
                    pass
            if len(localizations) > 1 and has_MTL==1:             # 'MTL' from localization.json
                if loc_MTL_names[pair_ct] != '' and loc_MTL_names[pair_ct] != ' ':
                    if str(loc_MTL_names[pair_ct]) != 'nan': # looking for "MTL" field in localizations.json
                        regs.append(loc_MTL_names[pair_ct].lower())
                        atlas_type.append('MTL_localization')
                        continue
                    else:
                        pass
                else:
                    pass
            if len(localizations) > 1:             # 'whole_brain' from localization.json
                if loc_wb_names[pair_ct] != '' and loc_wb_names[pair_ct] != ' ':
                    if str(loc_wb_names[pair_ct]) != 'nan': # looking for "MTL" field in localizations.json
                        regs.append(loc_wb_names[pair_ct].lower())
                        atlas_type.append('wb_localization')
                        continue
                    else:
                        pass
                else:
                    pass
            if 'wb.region' in pair_atlases:
                test_region = str(pair_atlases['wb.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region != 'nan'):
                    regs.append(test_region.lower())
                    atlas_type.append('wb')
                    continue
                else:
                    pass
            if len(localizations) > 1:             # 'dk' from localization.json
                if loc_dk_names[pair_ct] != '' and loc_dk_names[pair_ct] != ' ':
                    if str(loc_dk_names[pair_ct]) != 'nan': # looking for "dk" field in localizations.json
                        regs.append(loc_dk_names[pair_ct].lower())
                        atlas_type.append('dk_localization')
                        continue
                    else:
                        pass
                else:
                    pass
            if 'dk.region' in pair_atlases:
                test_region = str(pair_atlases['dk.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region != 'nan'):
                    regs.append(test_region.lower())
                    atlas_type.append('dk')
                    continue
                else:
                    pass
            if 'ind.corrected.region' in pair_atlases: # I don't think this ever has a region label but just in case
                test_region = str(pair_atlases['ind.corrected.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region not in 'nan'):
                    regs.append(test_region.lower())
                    atlas_type.append('ind.corrected')
                    continue
                else:
                    pass  
            if 'ind.region' in pair_atlases:
                test_region = str(pair_atlases['ind.region'])
                if (test_region is not None) and (len(test_region)>1) and \
                   (test_region not in 'None') and (test_region != 'nan'):
                    regs.append(test_region.lower())
                    atlas_type.append('ind')
                    # [tal_struct[i].atlases.ind.region for i in range(len(tal_struct))] # if you want to see ind atlases for comparison to above
                    # have to run this first though to work in pdb dubugger: globals().update(locals())                  
                    continue
                else:
                    regs.append('No atlas')
                    atlas_type.append('No atlas')
            else: 
                regs.append('No atlas')
                atlas_type.append('No atlas')
        except AttributeError:
            regs.append('error')
            atlas_type.append('error')
    return np.array(regs),np.array(atlas_type),np.array(pair_number),has_stein_das


def load_electrode_regions(protocol: str = 'all', rootdir: Optional[str] = None, experiments = [], 
                           cml_subject_args = [], session = 0, localization = 0, montage = 0, debug = False):
    # 2021-11-04 James Bruska
    
    # Get data and filter
    df = CMLReader.get_data_index(protocol, rootdir)
    
    if experiments:
        df = df[df["experiment"].isin(exps)]
    
    if debug:
        subs_not_present = 0
        for csa in cml_subject_args:
            if csa.subject not in df["subject"].unique():
                subs_not_present += 1
                protocol_str = "protocol ({}) ".format(protocol) if (protocol != "all") else ""
                rootdir_str = "rootdir ({}) ".format(rootdir) if (rootdir is not None) else ""
                experiments_str = "experiments ({}) ".format(experiments) if (experiments is not None) else ""
                output_str = protocol_str + rootdir_str + experiments_str
                print("Subject {} is not in the provided {}".format(csa.subject, output_str))
        print("There were {} subjects not in the loaded data".format(subs_not_present))
    
    return get_electrode_regions(df, cml_subject_args, session, localization, montage, debug, False)
    

@ignore_warnings
def get_electrode_regions(df, cml_subject_args = [], session = 0, localization = 0, montage = 0,
                          debug = False, debug_subs_not_present = True):
    # 2021-11-04 James Bruska
    
    # Group together by subject
    df_set = df.groupby(['subject']).agg(set).reset_index()
    
    # Inform of subjects that are not there
    if debug and debug_subs_not_present:
        subs_not_present = 0
        for csa in cml_subject_args:
            if csa.subject not in df["subject"].unique():
                subs_not_present += 1
                print("Subject {} is not in the provided data frame".format(csa.subject))
        print("There were {} subjects not in the provided data frame".format(subs_not_present))
    
    # Load the electrode regions for each subject
    elec_info = {"subject":[], "experiment":[], "localization":[], "montage":[], "session":[], 
                 "electrode region":[], "atlas type":[], "pair number":[], "has stein das":[]}
    num_succ_loading = 0
    num_failed_loading = 0
    for _, row in df_set.iterrows():
        sub = row["subject"]
        
        # Skip subjects not listed and get the localization and montage for subjects that are
        if cml_subject_args:
            if any(csa for csa in cml_subject_args if csa.subject == sub):
                _,loc,mon,sess,exp = next(csa for csa in cml_subject_args if csa.subject == sub)
            else:
                continue
        else:
            loc,mon,sess,exp = (localization, montage, session, "")
        
        # Load the electrode regions for the subject and add them to cumulative list
        try:
            reader = CMLReader(subject=sub, experiment=exp, session=sess, localization=loc, montage=mon)
            pairs = reader.load('pairs')
            try:
                localizations = reader.load('localization')
            except:
                localizations = []
            elec_regs,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs)
            
            elec_info["subject"].extend([sub]*pairs.shape[0])
            elec_info["experiment"].extend([exp]*pairs.shape[0])
            elec_info["localization"].extend([loc]*pairs.shape[0])
            elec_info["montage"].extend([mon]*pairs.shape[0])
            elec_info["session"].extend([sess]*pairs.shape[0])
            elec_info["electrode region"].extend(elec_regs)
            elec_info["atlas type"].extend(atlas_type)
            elec_info["pair number"].extend(pair_number)
            elec_info["has stein das"].extend([has_stein_das]*pairs.shape[0])
            
            num_succ_loading += 1
        except Exception as e:
            if debug:
                print(sub + " " + loc + " " + mon + " " + sess + " " + exp + " (" + str(row["experiment"]) + ") " + str(e))
            num_failed_loading += 1
    
    df_electrode_regions = pd.DataFrame(elec_info)
 
    # Print the number of loading successes and failures
    if debug:
        print("Num Electrode Regions Loaded: {}".format(df_electrode_regions.shape[0]))
        print("Num Subjects Succeeded To Load: {}/{}".format(num_succ_loading, df_set.shape[0]))
        print("Num Subjects Failed To Load: {}/{}".format(num_failed_loading, df_set.shape[0]))
    
    return df_electrode_regions

In [129]:
# 2021-11-04 James Bruska

import numpy as np
import pandas as pd

from cmlreaders import CMLReader
from typing import Optional
from dataclasses import dataclass

@dataclass
class CMLSubjectArgs:
    subject: str
    localization: int = 0
    montage: int = 0
    session: int = 1
    experiment: str = ""
    def __iter__(self):
        return iter((self.subject, self.localization, self.montage, self.session, self.experiment))

def load_cml_field(field, protocol: str = 'all', rootdir: Optional[str] = None, experiments = [],
                   cml_subject_args = [], session = 0, localization = 0, montage = 0, debug = False):
    # 2021-11-04 James Bruska
    
    # Get data and filter
    df = CMLReader.get_data_index(protocol, rootdir)
    
    if experiments:
        df = df[df["experiment"].isin(exps)]
    
    if debug:
        subs_not_present = 0
        for csa in cml_subject_args:
            if csa.subject not in df["subject"].unique():
                subs_not_present += 1
                protocol_str = "protocol ({}) ".format(protocol) if (protocol != "all") else ""
                rootdir_str = "rootdir ({}) ".format(rootdir) if (rootdir is not None) else ""
                experiments_str = "experiments ({}) ".format(experiments) if (experiments is not None) else ""
                output_str = protocol_str + rootdir_str + experiments_str
                print("Subject {} is not in the provided {}".format(csa.subject, output_str))
        print("There were {} subjects not in the loaded data".format(subs_not_present))
    
    return get_cml_field(df, field, cml_subject_args, session, localization, montage, debug, False)

@ignore_warnings
def get_cml_field(df, field, cml_subject_args = [], session = 0, localization = 0, montage = 0,
                  debug = True, debug_subs_not_present = True):
    # 2021-11-04 James Bruska
    
    # Group together by subject
    df_set = df.groupby(['subject']).agg(set).reset_index()
    
    # Inform of subjects that are not there
    if debug and debug_subs_not_present:
        subs_not_present = 0
        for csa in cml_subject_args:
            if csa.subject not in df["subject"].unique():
                subs_not_present += 1
                print("Subject {} is not in the provided data frame".format(csa.subject))
        print("There were {} subjects not in the provided data frame".format(subs_not_present))
    
    # Load the field entries for each subject
    entries = []
    num_succ_loading = 0
    num_failed_loading = 0
    for _, row in df_set.iterrows():
        sub = row["subject"]
        
        # Skip subjects not listed and get the localization and montage for subjects that are
        if cml_subject_args:
            if any(csa for csa in cml_subject_args if csa.subject == sub):
                _,loc,mon,sess,exp = next(csa for csa in cml_subject_args if csa.subject == sub)
            else:
                continue
        else:
            loc,mon,sess,exp = (localization, montage, session, "")
        
        # Load the field entries for the subject and add them to cumulative list
        try:
            sess_events = CMLReader(subject=sub, experiment=exp, session=sess, localization=loc, montage=mon).load(field)
            sess_events.insert(0, "subject", [sub]*sess_events.shape[0], True)  # Add subject name to dataframe
            sess_events.insert(1, "experiment", [exp]*sess_events.shape[0], True)  # Add experiment name to dataframe
            sess_events.insert(2, "localization", [loc]*sess_events.shape[0], True)  # Add localization to dataframe
            sess_events.insert(3, "montage", [mon]*sess_events.shape[0], True)  # Add montage to dataframe
            sess_events.insert(3, "session", [sess]*sess_events.shape[0], True)  # Add montage to dataframe
            num_succ_loading += 1
            entries.append(sess_events)
        except Exception as e:
            if debug:
                print(sub + " " + loc + " " + mon + " " + sess + " " + exp + " (" + str(row["experiment"]) + ") " + str(e))
            num_failed_loading += 1
     
    if entries:
        df_entries = pd.concat(entries)
    else:
        df_entries = pd.DataFrame({})
 
    # Print the number of loading successes and failures
    if debug:
        print("Num Entries: {}".format(df_entries.shape[0]))
        print("Num Succeeded To Load: {}/{}".format(num_succ_loading, df_set.shape[0]))
        print("Num Failed To Load: {}/{}".format(num_failed_loading, df_set.shape[0]))
    
    return df_entries


# Generate the electrode contacts

In [193]:
import numpy as np

exps = ['FR1']
ix = CMLReader.get_data_index("r1")
df = ix[ix["experiment"].isin(exps)]

subs = [CMLSubjectArgs("R1001P", 0, 0, 0), CMLSubjectArgs("R1350D", 0, 0, 0), CMLSubjectArgs("R2000F", 0, 0, 0)]
print(get_cml_field(df, "contacts", subs, True))

df = df[df["subject"] == "R1001P"]
print(get_cml_field(df, "contacts"))
#print(get_electrode_regions(df, session=0, localization=0, montage=0, debug=True))

Subject R2000F is not in the provided data frame
There were 1 subjects not in the provided data frame
Num Entries: 220
Num Succeeded To Load: 2/281
Num Failed To Load: 0/281
    subject experiment  localization  session  montage  contact    label  \
0    R1001P                        0        0        0        2     LAF1   
1    R1001P                        0        0        0        3     LAF2   
2    R1001P                        0        0        0        4     LAF3   
3    R1001P                        0        0        0        5     LAF4   
4    R1001P                        0        0        0        6     LAF5   
5    R1001P                        0        0        0        7     LAF6   
6    R1001P                        0        0        0        8     LPF1   
7    R1001P                        0        0        0        9     LPF2   
8    R1001P                        0        0        0       10     LPF3   
9    R1001P                        0        0        0       11   

In [192]:
brain_plot_settings = {
    "single_subject": False, # Single subject data
    "log10_data": False, # Take the log of all of the t_values for display
    "do_region_plot": True, # Plot colors based on regions rather than t_values
    "electrode_size": 3, # The size of the electrodes
    "colormap": "PuOr", # Override the default color scheme
#    "vmin": -3, # The min value of the t_values 
#    "vmax": 3, # The max value of the t_values
}

In [191]:
import numpy as np

pd.set_option("display.max_rows", None, "display.max_columns", None)

exps = ['RepFR1']
subs = ["R1590T"]
region_type = "mni.region"
#subs = ["R1001P"]
#region_type = "avg.region"

df = CMLReader.get_data_index("r1")
#df = df[df["experiment"].isin(exps)]
df = df[df["subject"].isin(subs)]

df_contacts = get_cml_field(df, "contacts")

df_e = df_contacts[["subject", "mni.x", "mni.y", "mni.z", "contact", region_type]]
df_e.loc[df_e[region_type].isnull(), region_type] = ""
df_e.replace([np.inf, -np.inf], np.nan)
df_e = df_e[~df_e.isnull().any(axis=1)]
df_e = df_e[~df_e.eq("NaN").any(axis=1)]

coords = df_e[["mni.x", "mni.y", "mni.z"]].values.tolist()
t_values = df_e[region_type].astype('category').cat.codes.apply(lambda x: (x+1)/len(values.unique())) # Region based t_values
regions = df_e[region_type].values.tolist()
#stim_coords = np.array([[-26.85, -20.37, -21.985]])
stim_coords = np.array([])
np.savez("brain_plot_data", coords=coords, t_values=t_values, regions=regions, stim_coords=stim_coords, settings=[brain_plot_settings])

There were 0 subjects not in the provided data frame
Num Entries: 126
Num Succeeded To Load: 1/1
Num Failed To Load: 0/1


/home1/jbruska/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [90]:
values = df_e["avg.region"].astype('category').cat.codes
values = values.apply(lambda x: (x+1)/len(values.unique()))